<h1>ETL</h1>

Comenzaremos importando nuestras librerias para comenzar el ETL

In [1]:
import gzip
import ast
import json
import pandas as pd
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd


Ahora crearemos un conjunto de funciones que nos ayudaran con los dataframes de pandas

Creación de DataFrames desde archivos JSON comprimidos: Las funciones crearDFJSON y crearDFAST permiten cargar datos desde archivos comprimidos en formato JSON, ya sea interpretando cada línea como un objeto JSON con json.loads() o como un diccionario de Python con ast.literal_eval().

Desanidación de datos: Las funciones desanidarDF y desanidar se encargan de desanidar datos contenidos en columnas de DataFrames. Esto es útil cuando se trabaja con datos JSON anidados, ya que permite expandir estructuras de datos complejas en un formato tabular más fácil de analizar.

In [2]:
def crearDFJSON(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(json.loads(line))
    f.close
    return pd.DataFrame(data)

def crearDFAST(ruta):
    data = []
    with gzip.open(ruta, 'rb') as f:
        for line in f:
            data.append(ast.literal_eval(line.decode('utf-8')))
    f.close
    return pd.DataFrame(data)

def desanidarDF(df, columna):
    df_desanidado = df.explode(columna)
    return df_desanidado

def desanidar(df, columna):
    dfDesanidado = pd.json_normalize(df[columna])
    df = df.reset_index(drop=True)
    df = pd.concat([df, dfDesanidado], axis=1)
    df = df.drop(columna, axis=1)
    return df



Creacion de los DataFrames

In [3]:
steamGamesDF = crearDFJSON('../data/steam_games.json.gz')
userReviews = crearDFAST('../data/user_reviews.json.gz')
usersItems = crearDFAST('../data/users_items.json.gz')

<h1>Steam Games</h1>

In [4]:
steamGamesDF

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Aqui creamos el DataFrame de los Nombres de los juegos que necesitaremos más adelante, borramos los duplicados, los vacios y nos quedamos solamente con la columna Titulo y ItemId

In [5]:
Nombres = steamGamesDF.iloc[:,[3,11]]
Nombres = Nombres.dropna()
Nombres = Nombres.drop_duplicates().reset_index(drop=True)
Nombres.columns = ['Titulo', 'ItemId']
Nombres.to_parquet('../data/NombresJuegos.parquet', index=False)

Eliminamos las columnas que no necesitemos para el DataFrame final, que es el que se usara para el entrenamiento del modelo ML

In [6]:
steamGamesDF = steamGamesDF.dropna().reset_index(drop=True)
columnas_elimnar = ['url', 'reviews_url', 'early_access', 'genres', 'app_name', 'developer', 'publisher', 'title', 'specs']
steamGamesDF = steamGamesDF.drop(columnas_elimnar, axis=1)
steamGamesDF.columns

Index(['release_date', 'tags', 'price', 'id'], dtype='object')

Rescatamos el año de la columna de la fecha de lanzamiento del videojuego

In [7]:
steamGamesDF['release_year'] = pd.to_datetime(steamGamesDF['release_date'], errors='coerce').dt.year
steamGamesDF = steamGamesDF.dropna(subset=['release_year'])
steamGamesDF=steamGamesDF.drop(['release_date'], axis=1).reset_index(drop=True)

Desanidamos la columna de los generos

In [8]:
steamGamesDF = desanidarDF(steamGamesDF, 'tags')
steamGamesDF = steamGamesDF.reset_index(drop=True)

In [9]:
steamGamesDF

,tags,price,id,release_year
0,Strategy,4.99,761140,2018.0
1,Action,4.99,761140,2018.0
2,Indie,4.99,761140,2018.0
3,Casual,4.99,761140,2018.0
4,Simulation,4.99,761140,2018.0
...,...,...,...,...
133474,Casual,4.99,658870,2017.0
133475,Puzzle,4.99,658870,2017.0
133476,Singleplayer,4.99,658870,2017.0
133477,Atmospheric,4.99,658870,2017.0


Ajustamos el precio de los juegos free to play a 0

In [10]:
steamGamesDF['price'] = steamGamesDF['price'].replace('Free To Play', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Free to Play', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Free', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Free Demo', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Play for Free!', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Install Now', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Play WARMACHINE: Tactics Demo', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Free Mod', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Play Now', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Free HITMAN™ Holiday Pack', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Play the Demo', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Third-party', 0)
steamGamesDF['price'] = steamGamesDF['price'].replace('Free Mod', 0)

/tmp/ipykernel_64109/1194919804.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  steamGamesDF['price'] = steamGamesDF['price'].replace('Third-party', 0)


In [11]:
steamGamesDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133479 entries, 0 to 133478
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tags          133479 non-null  object 
 1   price         133479 non-null  float64
 2   id            133479 non-null  object 
 3   release_year  133479 non-null  float64
dtypes: float64(2), object(2)
memory usage: 4.1+ MB


In [12]:
steamGamesDF

,tags,price,id,release_year
0,Strategy,4.99,761140,2018.0
1,Action,4.99,761140,2018.0
2,Indie,4.99,761140,2018.0
3,Casual,4.99,761140,2018.0
4,Simulation,4.99,761140,2018.0
...,...,...,...,...
133474,Casual,4.99,658870,2017.0
133475,Puzzle,4.99,658870,2017.0
133476,Singleplayer,4.99,658870,2017.0
133477,Atmospheric,4.99,658870,2017.0


Cambiamos de nombre a las columnas y borramos duplicados

In [13]:
steamGamesDF.columns = ['Etiquetas', 'Precio', 'ItemId', 'Año_Lanzamiento']
steamGamesDF = steamGamesDF.drop_duplicates()

Vemos las 30 etiquetas más usadas y las guardamos en una variable que usaremos más adelante

In [14]:
etiquetas_mas_comunes = steamGamesDF['Etiquetas'].value_counts().head(15)
etiquetas_mas_comunes

Etiquetas
Indie               13143
Action               9960
Adventure            7908
Casual               6849
Strategy             5564
Simulation           5057
RPG                  4101
Singleplayer         4028
Multiplayer          2133
Great Soundtrack     2057
Puzzle               1945
2D                   1866
Atmospheric          1771
Free to Play         1591
Platformer           1362
Name: count, dtype: int64

Reducimos el df con las etiquetas mas comunes

In [15]:
steamGamesDF = steamGamesDF[steamGamesDF['Etiquetas'].isin(etiquetas_mas_comunes.index)].reset_index(drop=True)
steamGamesDF

,Etiquetas,Precio,ItemId,Año_Lanzamiento
0,Strategy,4.99,761140,2018.0
1,Action,4.99,761140,2018.0
2,Indie,4.99,761140,2018.0
3,Casual,4.99,761140,2018.0
4,Simulation,4.99,761140,2018.0
...,...,...,...,...
69330,Indie,4.99,658870,2017.0
69331,Casual,4.99,658870,2017.0
69332,Puzzle,4.99,658870,2017.0
69333,Singleplayer,4.99,658870,2017.0


<h1>User Reviews</h1>

Desanidamos el dataframe

In [16]:
userReviews = desanidarDF(userReviews, 'reviews')
userReviews=desanidar(userReviews, 'reviews')

Eliminamos columnas que no nos sirven para el entrenamiento

In [17]:
columnas_elimnar = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'helpful']
userReviews = userReviews.drop(columnas_elimnar, axis=1).reset_index(drop=True)

Pasamos la columna de reviews por un proceso de separacion, 1 si es neutral, 2 si es positivo y 0 si es negativo su respectivo comentario

In [18]:
# Descargar los recursos necesarios para NLTK
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimientos
sid = SentimentIntensityAnalyzer()

# Función para asignar el sentimiento a cada reseña
def get_sentiment(review):
    if pd.isnull(review):
        return 1  # Valor neutral si no hay reseña
    else:
        scores = sid.polarity_scores(review)
        compound_score = scores['compound']
        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la columna 'sentiment_analysis'
userReviews['sentiment_analysis'] = userReviews['review'].apply(get_sentiment)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/konas/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
userReviews

,item_id,recommend,review,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,2
1,22200,True,It's unique and worth a playthrough.,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,251610,True,I know what you think when you see this title ...,2
4,227300,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...
59328,70,True,a must have classic from steam definitely wort...,2
59329,362890,True,this game is a perfect remake of the original ...,2
59330,273110,True,had so much fun plaing this and collecting res...,2
59331,730,True,:D,2


Eliminamos la columna de los reviews

In [20]:
userReviews = userReviews.drop(['review'], axis=1).reset_index(drop=True)

Cambiamos el nombre a las columnas

In [21]:
userReviews.columns = ['IdItem', 'Recomendado', 'Sentimiento']
userReviews = userReviews.dropna()

Eliminamos los duplicados

In [22]:
userReviews = userReviews.drop_duplicates().reset_index(drop=True)

In [23]:
userReviews

,IdItem,Recomendado,Sentimiento
0,1250,True,2
1,22200,True,2
2,43110,True,2
3,251610,True,2
4,227300,True,2
...,...,...,...
7878,369290,False,1
7879,209120,True,2
7880,220090,True,1
7881,262850,True,1


<h1>Users Items</h1>

In [24]:
usersItems

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Eliminamos columnas que no necesitamos

In [25]:
columnas_elimnar = ['items_count','steam_id', 'user_url']
usersItems = usersItems.drop(columnas_elimnar, axis=1).reset_index(drop=True)

Desanidamos la columna de items

In [26]:
usersItems = desanidarDF(usersItems, 'items')
usersItems = desanidar(usersItems, 'items')

Eliminamos los vacios y los duplicados

In [27]:
usersItems = usersItems.dropna()

In [28]:
usersItems = usersItems.drop_duplicates().reset_index(drop=True)

In [29]:
usersItems

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...
5094100,76561198329548331,346330,BrainBread 2,0.0,0.0
5094101,76561198329548331,373330,All Is Dust,0.0,0.0
5094102,76561198329548331,388490,One Way To Die: Steam Edition,3.0,3.0
5094103,76561198329548331,521570,You Have 10 Seconds 2,4.0,4.0


Eliminamos columnas

In [30]:
columnas_elimnar = ['playtime_2weeks', 'user_id', 'item_name']
usersItems = usersItems.drop(columnas_elimnar, axis=1).reset_index(drop=True)

Agrupamos los datos con la suma total del tiempo jugado de cada juego

In [31]:
usersItems = usersItems.groupby('item_id')['playtime_forever'].sum().reset_index()
usersItems

,item_id,playtime_forever
0,10,17107858.0
1,100,301732.0
2,10000,62685.0
3,1002,894.0
4,10040,0.0
...,...,...
10973,99890,127442.0
10974,9990,2083.0
10975,99900,17159071.0
10976,99910,426210.0


Cambiamos el nombre de las columnas

In [32]:
usersItems.columns = ["IdItem", "TiempoJugado"]

In [33]:
usersItems

,IdItem,TiempoJugado
0,10,17107858.0
1,100,301732.0
2,10000,62685.0
3,1002,894.0
4,10040,0.0
...,...,...
10973,99890,127442.0
10974,9990,2083.0
10975,99900,17159071.0
10976,99910,426210.0


<h1>Union</h2>

Unimos los tres dataframes por el id del item

In [34]:
df = pd.merge(steamGamesDF, userReviews, left_on='ItemId', right_on='IdItem', how='inner')
df = pd.merge(df, usersItems, on='IdItem', how='inner')

In [35]:
df.drop(columns=['IdItem'], inplace=True)

In [36]:
df

,Etiquetas,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado
0,Action,9.99,282010,1997.0,True,1,9319.0
1,Indie,9.99,282010,1997.0,True,1,9319.0
2,Multiplayer,9.99,282010,1997.0,True,1,9319.0
3,Singleplayer,9.99,282010,1997.0,True,1,9319.0
4,Action,9.99,70,1998.0,True,2,2650946.0
...,...,...,...,...,...,...,...
33539,Atmospheric,9.99,80,2004.0,True,0,2469131.0
33540,Simulation,9.99,80,2004.0,True,1,2469131.0
33541,Simulation,9.99,80,2004.0,False,1,2469131.0
33542,Simulation,9.99,80,2004.0,True,2,2469131.0


Obtenemos las variables dummys de la columna de etiquetas

In [37]:
# Obtener variables dummy para columnas específicas
df = pd.get_dummies(df, columns=['Etiquetas']).astype(int)
df = df.drop_duplicates()
df = df.dropna().reset_index(drop=True)

In [38]:
df

,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_2D,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Atmospheric,...,Etiquetas_Free to Play,Etiquetas_Great Soundtrack,Etiquetas_Indie,Etiquetas_Multiplayer,Etiquetas_Platformer,Etiquetas_Puzzle,Etiquetas_RPG,Etiquetas_Simulation,Etiquetas_Singleplayer,Etiquetas_Strategy
0,9,282010,1997,1,1,9319,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,282010,1997,1,1,9319,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,9,282010,1997,1,1,9319,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,9,282010,1997,1,1,9319,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,9,70,1998,1,2,2650946,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33539,9,80,2004,1,0,2469131,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
33540,9,80,2004,1,1,2469131,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
33541,9,80,2004,0,1,2469131,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
33542,9,80,2004,1,2,2469131,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Exportamos el dataframe como parquet

In [39]:
df.to_parquet('../data/ETL.parquet', index=False)